In [ ]:
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

# Suppressing warnings: Recbole-inherited warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
models = {
    'BPR': BPR,
    'LightGCN': LightGCN,
    'ItemKNN': ItemKNN,
    # 'NeuMF': NeuMF,
    # 'FM': FM,
    # 'DeepFM': DeepFM,
    # 'WideDeep': WideDeep,
    # 'KGCN': KGCN,
    # 'KGIN': KGIN,
    # 'KGAT': KGAT
}

In [ ]:
config_path = r"C:\Users\Sharjeel Mustafa\Documents\Documents\Academic\C874\C874-final-project\draft\config.yaml"
params_path = r"C:\Users\Sharjeel Mustafa\Documents\Documents\Academic\C874\C874-final-project\draft\model.hyper"

In [ ]:
import ray
ray.init()

In [ ]:
results = {}
for model_name, model_class in models.items():
    print(f"\033[93mRunning model: {model_name}\033[0m")
    # load configuration
    config = Config(model=model_name, dataset='ml-100k')
    config_dict = {'model': model_name, 'dataset': 'ml-100k'}

    # Hyperparameter tuning setup
    hp = HyperTuning(
        objective_function=objective_function,
        algo='exhaustive',
        early_stop=10,
        max_evals=100,
        params_file=params_path,
        fixed_config_file_list=[config_path]
    )
    hp.run()
    hp.export_result(output_file='hyper_example.result')
    print('best params: ', hp.best_params)
    print('best result: ')
    print(hp.params2result[hp.params2str(hp.best_params)])

    # Merge config with best hyperparameters, initialize seed
    config = {**config_dict, **hp.best_params}
    config = Config(model=model_name, dataset='ml-100k', config_dict=config)
    init_seed(config['seed'], config['reproducibility'])

    # Dataset loading and preparation
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Model initialization
    model = model_class(config, train_data._dataset).to(config['device'])

    # Trainer initialization and training
    trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
    trainer = trainer_class(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

    # Evaluation
    test_result = trainer.evaluate(test_data)
    results[model_name] = test_result

In [ ]:
for model_name, test_result in results.items():
    print(f"\n\033[92mResults for {model_name}:\033[0m")
    for key, value in test_result.items(): print(f"{key}: {value}")

In [11]:
from recbole.quick_start import run_recbole

run_recbole(model='LightGCN', dataset='ml-100k')

Evaluate   :   0%|                                                          | 0/472 [00:00<?, ?it/s]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|                                  | 0/472 [00:00<?, ?it/s, GPU RAM: 0.06 G/8.00 G]:   0%|   

{'best_valid_score': 0.3162,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.1626),
              ('mrr@10', 0.3162),
              ('ndcg@10', 0.1765),
              ('hit@10', 0.6394),
              ('precision@10', 0.1205)]),
 'test_result': OrderedDict([('recall@10', 0.1853),
              ('mrr@10', 0.376),
              ('ndcg@10', 0.2116),
              ('hit@10', 0.6819),
              ('precision@10', 0.1406)])}